In [1]:
import numpy as np
import pandas as pd
import os
import os.path
import datetime
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import plotly.offline as py
!pip install cufflinks
import cufflinks as cf
!pip install plotly==4.9.0
import plotly.graph_objects as go

In [2]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [3]:
#DataFrame de Activos ciudad de Medellín
df = pd.read_csv(os.path.join('../Output', 'data_medellin.csv'))
df = df[['FECHA','ACTIVOS']]
df2 = pd.to_datetime(df['FECHA'])
df.index = df2
data_activos = df.drop(['FECHA'], axis=1)
data_activos

,ACTIVOS
FECHA,
2020-03-09,1.0
2020-03-11,3.0
2020-03-14,5.0
2020-03-15,5.0
2020-03-19,8.0
...,...
2020-08-29,11282.0
2020-08-30,10904.0
2020-08-31,10986.0


In [4]:
#MODELO SIR PARA VARIOS BETA Y GAMMA

In [29]:
def modelo_SIR(beta,gamma,t):
    N=100000
    I0, R0=1, 0
    S0=N - I0 - R0
    
    # Las ecuaciones diferenciales del modelo SIR
    def deriv(y, t, N, beta, gamma):
        S, I, R = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I
        dRdt = gamma * I
        return dSdt, dIdt, dRdt

    # Vector de las condiciones iniciales
    y0 = S0, I0, R0
    # Resolver el sistema de ecuaciones diferenciales, en la secuencia de días que ya definimos
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T
    return (I)

beta_x = 0.01
gamma_x = 0.015
mse_x = 1000000000000

real=data_activos['ACTIVOS']
t = np.linspace(0, len(data_activos), len(data_activos))


for beta in np.arange(0.01,0.3,0.01):
    for gamma in np.arange(0.015,0.3,0.005):
        I = modelo_SIR(beta,gamma,t)
        mse = mean_squared_error(real,I)
        if mse < mse_x:
            beta_x = beta
            gamma_x = gamma
            mse_x = mse
print(beta_x,gamma_x,mse_x)

0.16 0.085 1090964.4422836746


In [30]:
N =   100000
I0, R0 = 1, 0
S0 = N - I0 - R0
beta, gamma = beta_x, gamma_x

t1 = np.linspace(0, len(data_activos)+90, len(data_activos)+90)

def deriv(y, t1, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

y0 = S0, I0, R0

ret = odeint(deriv, y0, t1, args=(N, beta, gamma))
S, I, R = ret.T


fig = go.Figure()
fig.add_trace(go.Scatter(x=t1, y=S,mode='lines',name='Susceptible'))
fig.add_trace(go.Scatter(x=t1, y=R,mode='lines',name='Recuperada y fallecidos'))
fig.add_trace(go.Scatter(x=t1, y=I,mode='lines',name='Infectada'))
fig.add_trace(go.Scatter(x=t, y=data_activos['ACTIVOS'],mode='lines',name='Activos Reales'))
fig.show()